# Simulation for non-contextual settings
We adjusted the simulation file provided by [Hadad, Vitor, et al. (2021)](https://arxiv.org/abs/1911.02768) to only contains the methods of interest in our banditsCI package: uniform and two-point.

The original simulation code provided by [Hadad, Vitor, et al. (2021)](https://arxiv.org/abs/1911.02768) can be accessed [here](https://github.com/gsbDBI/adaptive-confidence-intervals/blob/master/experiments/main/simulations.ipynb).

In [7]:

import sys
import subprocess
import pickle
import os
import numpy as np
import pandas as pd

from time import time
from sys import argv
from random import choice
from time import time
from os.path import dirname, realpath, join, exists
from os import makedirs, chmod
from getpass import getuser

from adaptive_CI.experiments import run_mab_experiment
from adaptive_CI.compute import stick_breaking
from adaptive_CI.inference import *
from adaptive_CI.weights import *

%reload_ext autoreload
%autoreload 2

In [8]:
start_time = time()

In [9]:
def on_sherlock():
    """ 
    Note: This can be ignored by non-Stanford members.

    Checks if running on Stanford's Sherlock cluster
    """
    return 'GROUP_SCRATCH' in os.environ


def get_sherlock_dir(project, *tail, create=True):
    """
    Note: This can be ignored by non-Stanford members.
    
    Output consistent folder name in Sherlock.
    If create=True and on Sherlock, also makes folder with group permissions.
    If create=True and not on Sherlock, does not create anything.

    '/scratch/groups/athey/username/project/tail1/tail2/.../tailn'.

    >>> get_sherlock_dir('adaptive-inference')
    '/scratch/groups/athey/adaptive-inference/vitorh'

    >>> get_sherlock_dir('toronto')
    '/scratch/groups/athey/toronto/vitorh/'

    >>> get_sherlock_dir('adaptive-inference', 'experiments', 'exp_out')
    '/scratch/groups/athey/adaptive-inference/vitorh/experiments/exp_out'
    """
    base = join("/", "scratch", "groups", "athey", project, getuser())
    path = join(base, *tail)
    if not exists(path) and create and on_sherlock():
        makedirs(path, exist_ok=True)
        # Correct permissions for the whole directory branch
        chmod_path = base
        chmod(base, 0o775)
        for child in tail:
            chmod_path = join(chmod_path, child)
            chmod(chmod_path, 0o775)
    return path


def compose_filename(prefix, extension):
    """
    Creates a unique filename based on Github commit id and time.
    Useful when running in parallel on server.

    INPUT:
        - prefix: file name prefix
        - extension: file extension

    OUTPUT:
        - fname: unique filename
    """
    # Tries to find a commit hash
    try:
        commit = subprocess\
            .check_output(['git', 'rev-parse', '--short', 'HEAD'],
                          stderr=subprocess.DEVNULL)\
            .strip()\
            .decode('ascii')
    except subprocess.CalledProcessError:
        commit = ''

    # Other unique identifiers
    rnd = str(int(time() * 1e8 % 1e8))
    sid = tid = jid = ''
    ident = filter(None, [prefix, commit, jid, sid, tid, rnd])
    basename = "_".join(ident)
    fname = f"{basename}.{extension}"
    return fname

In [10]:
num_sims = 200 if on_sherlock() else 1

# DGP specification
# ----------------------------------------------------
noise_func = 'uniform'
truths = {
    'nosignal': np.array([1., 1., 1.]),
    'lowSNR': np.array([.9, 1., 1.1]),
    'highSNR': np.array([.5, 1., 1.5])
}
# if on_sherlock():
#     Ts = [1_000, 5_000, 10_000, 50_000, 100_000]
# else:
#     Ts = [5_000]
# Setting Ts to always be [5000] regardless of the environment
Ts = [5_000]
floor_decays = [.7]
initial = 5  # initial number of samples of each arm to do pure exploration
exploration = 'TS'
noise_scale = 1.

In [11]:
df_stats = []
df_lambdas = []

In [12]:
# Run simulations
for s in range(num_sims):
    if (s+1) % 10 == 0:
        print(f'Running simulation {s+1}/{num_sims}')

    """ Experiment configuration """
    T = choice(Ts)  # number of samples
    experiment = choice(list(truths.keys()))
    truth = truths[experiment]
    K = len(truth)  # number of arms
    floor_start = 1/K
    floor_decay = choice(floor_decays)

    """ Generate data """
    noise = np.random.uniform(-noise_scale, noise_scale, size=(T, K))
    ys = truth + noise

    """ Run experiment """
    data = run_mab_experiment(
        ys,
        initial=initial,
        floor_start=floor_start,
        floor_decay=floor_decay,
        exploration=exploration)

    probs = data['probs']
    rewards = data['rewards']
    arms = data['arms']

    """ Compute AIPW scores """
    muhat = np.row_stack([np.zeros(K), sample_mean(rewards, arms, K)[:-1]])
    scores = aw_scores(rewards, arms, probs, muhat)

    """ Compute weights """
    # Two-point allocation rate
    twopoint_ratio = twopoint_stable_var_ratio(e=probs, alpha=floor_decay)
    twopoint_ratio_old = twopoint_stable_var_ratio_old(probs, floor_start, floor_decay)
    twopoint_h2es = stick_breaking(twopoint_ratio)
    twopoint_h2es_old = stick_breaking(twopoint_ratio_old)
    wts_twopoint = np.sqrt(np.maximum(0., twopoint_h2es * probs))

    # Other weights: lvdl(constant allocation rate), propscore and uniform
    wts_lvdl = np.sqrt(probs)
    wts_propscore = probs
    wts_uniform = np.ones_like(probs)

    """ Estimate arm values """
    # for each weighting scheme, return [estimate, S.E, bias, 90%-coverage, t-stat, mse, truth]
    stats = dict(
        uniform=evaluate_aipw_stats(scores, wts_uniform, truth),
        propscore=evaluate_aipw_stats(scores, wts_propscore, truth),
        lvdl=evaluate_aipw_stats(scores, wts_lvdl, truth),
        two_point=evaluate_aipw_stats(scores, wts_twopoint, truth),
        beta_bernoulli=evaluate_beta_bernoulli_stats(rewards, arms, truth, K, floor_decay, alpha=.1),
        gamma_exponential=evaluate_gamma_exponential_stats(rewards, arms, truth, K, floor_decay, c=2, expected_noise_variance=1/3, alpha=.1),
        sample_mean_naive=evaluate_sample_mean_naive_stats(rewards, arms, truth, K, alpha=.1)
    )
    
    # # add estimates of W_decorrelation
    W_name = f'wdecorr_results/W_lambdas_{experiment}-{noise_func}-{T}-{floor_decay}.npz'
    try:
        W_save = np.load(W_name)  # load presaved W-lambdas
        for percentile, W_lambda in zip(W_save['percentiles'], W_save['W_lambdas']):
            stats[f'W-decorrelation_{percentile}'] = wdecorr_stats(arms, rewards, K, W_lambda, truth)
    except FileNotFoundError:
        print(f'Could not find relevant w-decorrelation file {W_name}. Ignoring.')
        
    
    """ Estimate contrasts """
    contrasts = dict(
        uniform=evaluate_aipw_contrasts(scores, wts_uniform, truth),
        propscore=evaluate_aipw_contrasts(scores, wts_propscore, truth),
        lvdl=evaluate_aipw_contrasts(scores, wts_lvdl, truth),
        two_point=evaluate_aipw_contrasts(scores, wts_twopoint, truth),
        beta_bernoulli=evaluate_beta_bernoulli_contrasts(rewards, arms, truth, K, floor_decay, alpha=.1),
        gamma_exponential=evaluate_gamma_exponential_contrasts(rewards, arms, truth, K, floor_decay, c=2, expected_noise_variance=1/3, alpha=.1),
        sample_mean_naive=evaluate_sample_mean_naive_contrasts(rewards, arms, truth, K, alpha=.1)
    )
# have a look at `contrast` dataframe
#     print(contrasts)
    
    """ Save results """
    config = dict(
        T=T,
        K=K,
        noise_func=noise_func,
        noise_scale=noise_scale,
        floor_start=floor_start,
        floor_decay=floor_decay,
        initial=initial,
        dgp=experiment,
    )

    ratios = dict(
        lvdl=np.ones((T, K)) / np.arange(T, 0, -1)[:, np.newaxis],
        two_point=twopoint_ratio,
    )
    
    # save lambda values at selected timepoints
    saved_timepoints = list(range(0, T, T // 250)) + [T-1]
    for ratio in ratios:
        ratios[ratio] = ratios[ratio][saved_timepoints, :]
    
    # tabulate arm values
    tabs_stats = []
    for method, stat in stats.items():
        tab_stats = pd.DataFrame({"statistic": ["estimate", "stderr", "bias", "90% coverage of t-stat", "t-stat", "mse", "CI_width", "truth"] * stat.shape[1],
                                  "policy": np.repeat(np.arange(K), stat.shape[0]),
                                  "value":  stat.flatten(order='F'),
                                  "method": method,
                                 **config})
        tabs_stats.append(tab_stats)


    # tabulate arm contrasts
    tabs_contrasts = []
    for method, contrast in contrasts.items():
        tabs_contrast = pd.DataFrame({"statistic": ["estimate", "stderr", "bias", "90% coverage of t-stat", "t-stat", "mse", "CI_width", "truth"] * contrast.shape[1],
                                      "policy": np.repeat([f"(0,{k})" for k in np.arange(1, K)], contrast.shape[0]),
                                      "value": contrast.flatten(order='F'),
                                      "method": method,
                                     **config})
        tabs_contrasts.append(tabs_contrast)

    
    df_stats.extend(tabs_stats)
    df_stats.extend(tabs_contrasts)
    
    
    """ Save relevant lambda weights, if applicable """
    if T == max(Ts):
        saved_timepoints = list(range(0, T, T // 500))
        lambdas = twopoint_ratio[saved_timepoints] * (T - np.array(saved_timepoints)[:,np.newaxis])
        lambdas = {key: value for key, value in enumerate(lambdas.T)}
        dfl = pd.DataFrame({**lambdas, **config, 'time': saved_timepoints})
        dfl = pd.melt(dfl, id_vars=list(config.keys()) + ['time'], var_name='policy', value_vars=list(range(K)))
        df_lambdas.append(dfl)
        
    print(f"Time passed {time()-start_time}s")

Time passed 2.34184193611145s


/var/folders/vw/_ffz0lz95v93q6_6s4x_60840000gp/T/ipykernel_7108/907649318.py:31: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  muhat = np.row_stack([np.zeros(K), sample_mean(rewards, arms, K)[:-1]])


----

In [40]:
evaluate_aipw_stats(scores, wts_uniform, truth)

array([[ 9.58950795e-01,  1.00674262e+00,  1.15818710e+00],
       [ 1.30935706e-01,  8.49990467e-03,  2.93785161e-01],
       [-4.10492053e-02,  6.74262327e-03,  1.58187102e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [-3.13506580e-01,  7.93258693e-01,  5.38444835e-01],
       [ 1.68503726e-03,  4.54629686e-05,  2.50231593e-02],
       [ 2.15370071e-01,  1.39810990e-02,  4.83233587e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00]])

In [44]:
df_stats.loc[(df_stats['method'] == "uniform") & 
(df_stats['statistic'].isin(['estimate', 'stderr']))]

,statistic,policy,value,method,T,K,noise_func,noise_scale,floor_start,floor_decay,initial,dgp
0,estimate,0,0.958951,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
1,stderr,0,0.130936,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
8,estimate,1,1.006743,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
9,stderr,1,0.008500,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
16,estimate,2,1.158187,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
17,stderr,2,0.293785,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
264,estimate,"(0,1)",0.199236,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
265,stderr,"(0,1)",0.321644,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
272,estimate,"(0,2)",0.151444,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
273,stderr,"(0,2)",0.293907,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal


In [48]:
df_stats.loc[(df_stats['method'] == "two_point") & 
(df_stats['statistic'].isin(['estimate', 'stderr']))]

,statistic,policy,value,method,T,K,noise_func,noise_scale,floor_start,floor_decay,initial,dgp
72,estimate,0,0.944510,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
73,stderr,0,0.044391,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
80,estimate,1,1.008370,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
81,stderr,1,0.008846,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
88,estimate,2,0.839830,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
89,stderr,2,0.154111,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
312,estimate,"(0,1)",-0.104679,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
313,stderr,"(0,1)",0.160960,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
320,estimate,"(0,2)",-0.168539,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
321,stderr,"(0,2)",0.154374,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal


In [13]:
print(contrasts)

{'uniform': array([[0.19923631, 0.15144448],
       [0.32164373, 0.29390713],
       [0.19923631, 0.15144448],
       [1.        , 1.        ],
       [0.61943166, 0.51528005],
       [0.03969511, 0.02293543],
       [0.52905685, 0.48343421],
       [0.        , 0.        ]]), 'propscore': array([[-0.12954903, -0.17613859],
       [ 0.09915868,  0.09505471],
       [-0.12954903, -0.17613859],
       [ 1.        ,  0.        ],
       [-1.30648196, -1.85302316],
       [ 0.01678295,  0.0310248 ],
       [ 0.16310152,  0.15635109],
       [ 0.        ,  0.        ]]), 'lvdl': array([[ 2.22629695e-02, -1.64844384e-02],
       [ 1.88334682e-01,  1.84278158e-01],
       [ 2.22629695e-02, -1.64844384e-02],
       [ 1.00000000e+00,  1.00000000e+00],
       [ 1.18209611e-01, -8.94541090e-02],
       [ 4.95639811e-04,  2.71736710e-04],
       [ 3.09782984e-01,  3.03110596e-01],
       [ 0.00000000e+00,  0.00000000e+00]]), 'two_point': array([[-0.10467946, -0.16853925],
       [ 0.16096009,  0.1

In [14]:
tabs_contrasts

[                 statistic policy     value   method     T  K noise_func  \
 0                 estimate  (0,1)  0.199236  uniform  5000  3    uniform   
 1                   stderr  (0,1)  0.321644  uniform  5000  3    uniform   
 2                     bias  (0,1)  0.199236  uniform  5000  3    uniform   
 3   90% coverage of t-stat  (0,1)  1.000000  uniform  5000  3    uniform   
 4                   t-stat  (0,1)  0.619432  uniform  5000  3    uniform   
 5                      mse  (0,1)  0.039695  uniform  5000  3    uniform   
 6                 CI_width  (0,1)  0.529057  uniform  5000  3    uniform   
 7                    truth  (0,1)  0.000000  uniform  5000  3    uniform   
 8                 estimate  (0,2)  0.151444  uniform  5000  3    uniform   
 9                   stderr  (0,2)  0.293907  uniform  5000  3    uniform   
 10                    bias  (0,2)  0.151444  uniform  5000  3    uniform   
 11  90% coverage of t-stat  (0,2)  1.000000  uniform  5000  3    uniform   

### Saving

Save the `scores` and `probs` generated from the experiment. We will use them with banditsCI R package to compare the estimates and standard errors

In [15]:
np.savetxt("./results/scores.csv", scores, delimiter=",")
np.savetxt("./results/probs.csv", probs, delimiter=",")

In [16]:
if on_sherlock():
    write_dir = get_sherlock_dir('adaptive-confidence-intervals', 'simulations', create=True)
    print(f"saving at {write_dir}")
else:
     write_dir = join(os.getcwd(), 'results')

In [17]:
df_stats = pd.concat(df_stats, ignore_index=True, sort=False)

Saving information about contrasts.

In [18]:
filename_contrast = compose_filename(f'contrast', 'pkl')
write_path_contrast = os.path.join(write_dir, filename_contrast)

df_contrast = df_stats.query(
    '(policy == "(0,1)" or policy == "(0,2)") and '
    'statistic in ["estimate", "stderr", "mse", "bias", "CI_width"] and '
    "method in ['uniform', 'two_point', 'sample_mean_naive']")
df_contrast.to_pickle(write_path_contrast)

In [19]:
contrasts

{'uniform': array([[0.19923631, 0.15144448],
        [0.32164373, 0.29390713],
        [0.19923631, 0.15144448],
        [1.        , 1.        ],
        [0.61943166, 0.51528005],
        [0.03969511, 0.02293543],
        [0.52905685, 0.48343421],
        [0.        , 0.        ]]),
 'propscore': array([[-0.12954903, -0.17613859],
        [ 0.09915868,  0.09505471],
        [-0.12954903, -0.17613859],
        [ 1.        ,  0.        ],
        [-1.30648196, -1.85302316],
        [ 0.01678295,  0.0310248 ],
        [ 0.16310152,  0.15635109],
        [ 0.        ,  0.        ]]),
 'lvdl': array([[ 2.22629695e-02, -1.64844384e-02],
        [ 1.88334682e-01,  1.84278158e-01],
        [ 2.22629695e-02, -1.64844384e-02],
        [ 1.00000000e+00,  1.00000000e+00],
        [ 1.18209611e-01, -8.94541090e-02],
        [ 4.95639811e-04,  2.71736710e-04],
        [ 3.09782984e-01,  3.03110596e-01],
        [ 0.00000000e+00,  0.00000000e+00]]),
 'two_point': array([[-0.10467946, -0.16853925],
 

In [20]:
import pandas as pd
# Reading the pickle file into a DataFrame
df_contrast = pd.read_pickle(write_path_contrast)

# Printing the DataFrame
print(df_contrast)

    statistic policy     value             method     T  K noise_func  \
264  estimate  (0,1)  0.199236            uniform  5000  3    uniform   
265    stderr  (0,1)  0.321644            uniform  5000  3    uniform   
266      bias  (0,1)  0.199236            uniform  5000  3    uniform   
269       mse  (0,1)  0.039695            uniform  5000  3    uniform   
270  CI_width  (0,1)  0.529057            uniform  5000  3    uniform   
272  estimate  (0,2)  0.151444            uniform  5000  3    uniform   
273    stderr  (0,2)  0.293907            uniform  5000  3    uniform   
274      bias  (0,2)  0.151444            uniform  5000  3    uniform   
277       mse  (0,2)  0.022935            uniform  5000  3    uniform   
278  CI_width  (0,2)  0.483434            uniform  5000  3    uniform   
312  estimate  (0,1) -0.104679          two_point  5000  3    uniform   
313    stderr  (0,1)  0.160960          two_point  5000  3    uniform   
314      bias  (0,1) -0.104679          two_point  

## Compare the estimate and std.error from uniform and two_point

In [14]:
import pandas as pd

# Filter the rows where the statistic is either 'estimate' or 'stderr' and method is 'uniform' or 'two_point'
filtered_df = df_contrast.query("statistic in ['estimate', 'stderr'] and method in ['uniform', 'two_point']")

# Pivot the table to get 'estimate' and 'stderr' as columns
pivot_df = filtered_df.pivot_table(index=['method', 'policy'], columns='statistic', values='value', aggfunc='first').reset_index()

# Rename the columns appropriately
comparison_df = pivot_df.rename(columns={'estimate': 'estimate', 'stderr': 'std.error'})

# Filter to only include the rows for 'uniform' and 'two_point' methods
comparison_df = comparison_df[comparison_df['method'].isin(['uniform', 'two_point'])]

# Display the DataFrame
print(comparison_df)


statistic     method policy  estimate  std.error
0          two_point  (0,1)  0.339118   0.111285
1          two_point  (0,2)  0.079033   0.062857
2            uniform  (0,1)  0.618107   0.187946
3            uniform  (0,2) -0.081485   0.146073


In [15]:
tabs_stats

[                 statistic  policy     value   method     T  K noise_func  \
 0                 estimate       0  0.478578  uniform  5000  3    uniform   
 1                   stderr       0  0.187762  uniform  5000  3    uniform   
 2                     bias       0 -0.421422  uniform  5000  3    uniform   
 3   90% coverage of t-stat       0  0.000000  uniform  5000  3    uniform   
 4                   t-stat       0 -2.244449  uniform  5000  3    uniform   
 5                      mse       0  0.177597  uniform  5000  3    uniform   
 6                 CI_width       0  0.308841  uniform  5000  3    uniform   
 7                    truth       0  0.900000  uniform  5000  3    uniform   
 8                 estimate       1  1.178170  uniform  5000  3    uniform   
 9                   stderr       1  0.145833  uniform  5000  3    uniform   
 10                    bias       1  0.178170  uniform  5000  3    uniform   
 11  90% coverage of t-stat       1  1.000000  uniform  5000  3 

In [27]:
scores[:,1].mean()-scores[:,0].mean()

0.6995919830469092

In [28]:
scores[:,2].mean()-scores[:,0].mean()

0.6181070800711507

In [41]:
evalwts = wts_uniform
alpha=.1

In [42]:
arm_estimate = np.sum(evalwts * scores, 0) / np.sum(evalwts, 0)

In [43]:
arm_estimate

array([0.47857766, 1.17816965, 1.09668474])

In [45]:
scores.mean(axis = 0)

array([0.47857766, 1.17816965, 1.09668474])

In [46]:
estimate = arm_estimate[-1] - arm_estimate[:-1]
estimate

array([ 0.61810708, -0.0814849 ])

In [47]:
arm_estimate[-1]

1.0966847434186828

In [48]:
arm_estimate[:-1]

array([0.47857766, 1.17816965])